In [35]:
import zipfile
import os

zip_file_path = '/content/drive/MyDrive/archive.zip'
extract_path = '/content/drive/MyDrive/pest_classifier'


os.makedirs(extract_path, exist_ok=True)


check_path = os.path.join(extract_path, 'classification')

if os.path.exists(check_path) and os.listdir(check_path):
    print(f"Files already seem to be extracted to: {extract_path}")
else:
    print(f"Extracting files to: {extract_path}")
    try:

        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print(f"Successfully extracted files to: {extract_path}")
    except FileNotFoundError:
        print(f"Error: Zip file not found at {zip_file_path}. Please check the path.")
    except Exception as e:
        print(f"An error occurred during extraction: {e}")

Extracting files to: /content/drive/MyDrive/pest_classifier
Error: Zip file not found at /content/drive/MyDrive/archive.zip. Please check the path.


In [8]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: Mountpoint must not already contain files

In [9]:
import gc
gc.enable()

In [11]:
import os
import cv2
import numpy as np
import pathlib
import gc

base_classification_dir = pathlib.Path("/content/drive/MyDrive/pest_classifier/classification")

# Select only 5 categories for training
selected_category_names = [
    "rice leaf roller",
    "rice leaf caterpillar",
    "paddy stem maggot",
    "asiatic rice borer",
    "yellow rice borer"
]

directory_names = [str(i) for i in range(0, 102)]

train_dir = base_classification_dir / "train"
validation_dir = base_classification_dir / "val"
test_dir = base_classification_dir / "test"


print("\nDataset Paths Defined.")
print(f"Training data path: {train_dir}")
print(f"Validation data path: {validation_dir}")
print(f"Test data path: {test_dir}")


Dataset Paths Defined.
Training data path: /content/drive/MyDrive/pest_classifier/classification/train
Validation data path: /content/drive/MyDrive/pest_classifier/classification/val
Test data path: /content/drive/MyDrive/pest_classifier/classification/test


In [ ]:
import psutil
print(f"Memory used: {psutil.virtual_memory().percent}%")


Memory used: 15.9%


In [34]:
import os

train_dir = "/content/drive/MyDrive/pest_classifier/classification/train"
folders = sorted(os.listdir(train_dir))
print("Folders found:", folders)
print("Total folders:", len(folders))


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/pest_classifier/classification/train'

In [20]:
# Define image size and batch size
img_size = 224
batch_size = 32

print(f"Image size set to: {img_size}x{img_size}")
print(f"Batch size set to: {batch_size}")

Image size set to: 224x224
Batch size set to: 32


In [28]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50, MobileNetV2, InceptionV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

## Define resnet model

### Subtask:
Build a ResNet model using pre-trained weights (e.g., ResNet50, ResNet101) and add custom layers for your classification task.


**Reasoning**:
Build the ResNet model according to the instructions.



In [27]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50

base_resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# Freeze the base model layers
for layer in base_resnet_model.layers:
    layer.trainable = False

x = base_resnet_model.output

x = layers.MaxPooling2D(pool_size=(2, 2))(x)

x = layers.Dense(512, activation='relu')(x)

x = layers.Dropout(0.5)(x)

# Update the output layer to have the number of selected categories
predictions = layers.Dense(len(selected_category_names), activation='softmax')(x)

resnet_model = models.Model(inputs=base_resnet_model.input, outputs=predictions)

resnet_model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_2[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 24,639,365 (93.99 MB)

 Trainable params: 1,051,653 (4.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

## Define mobilenet model

### Subtask:
Build a MobileNet model using pre-trained weights (e.g., MobileNetV2) and add custom layers for your classification task.


**Reasoning**:
Build the MobileNetV2 model by loading pre-trained weights, freezing the base layers, and adding custom layers for classification, then print the model summary.



In [26]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2

base_mobilenet_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

for layer in base_mobilenet_model.layers:
    layer.trainable = False


x = base_mobilenet_model.output

x = layers.GlobalAveragePooling2D()(x)

x = layers.Dense(512, activation='relu')(x)

x = layers.Dropout(0.5)(x)

# Update the output layer to have the number of selected categories
predictions = layers.Dense(len(selected_category_names), activation='softmax')(x)


mobilenet_model = models.Model(inputs=base_mobilenet_model.input, outputs=predictions)


mobilenet_model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,916,421 (11.13 MB)

 Trainable params: 658,437 (2.51 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

## Define inceptionv3 model

### Subtask:
Build an InceptionV3 model using pre-trained weights and add custom layers for your classification task.


**Reasoning**:
Build the InceptionV3 model by loading the pre-trained weights, adding custom layers, and printing the summary.



In [24]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define image size
img_size = 224

base_inceptionv3_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

for layer in base_inceptionv3_model.layers:
    layer.trainable = False

x = base_inceptionv3_model.output

x = layers.GlobalAveragePooling2D()(x)

x = layers.Dense(512, activation='relu')(x)

x = layers.Dropout(0.5)(x)

# Update the output layer to have the number of selected categories
predictions = layers.Dense(len(selected_category_names), activation='softmax')(x)

inceptionv3_model = models.Model(inputs=base_inceptionv3_model.input, outputs=predictions)

inceptionv3_model.summary()

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 111, 111,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 111, 111,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 109, 109,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 109, 109,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 54, 54,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 54, 54,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 52, 52,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 22,854,437 (87.18 MB)

 Trainable params: 1,051,653 (4.01 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [29]:

resnet_model.compile(optimizer=Adam(learning_rate=0.001),
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

mobilenet_model.compile(optimizer=Adam(learning_rate=0.001),
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

inceptionv3_model.compile(optimizer=Adam(learning_rate=0.001),
                          loss='sparse_categorical_crossentropy',
                          metrics=['accuracy'])

print("Models compiled successfully.")

Models compiled successfully.


In [30]:

from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')
print("Mixed precision enabled.")

Mixed precision enabled.


In [33]:
import tensorflow as tf
import os

def load_and_preprocess_image(image_path, label):
    """Loads and preprocesses an image."""
    # Ensure image_path is treated as a string
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [img_size, img_size])
    img = img / 255.0
    return img, label

def create_dataset(directory, category_names, shuffle=True):
    """Creates a tf.data.Dataset from image files in a directory."""
    image_paths = []
    labels = []
    # Define the mapping from category name to original index based on the full list
    full_directory_names = [str(i) for i in range(0, 102)]
    category_to_original_index = {name: full_directory_names.index(str(i)) for i, name in enumerate(selected_category_names)}


    for i, category in enumerate(category_names):
        # Map the selected category name back to its original index
        if category in category_to_original_index:
            original_index = category_to_original_index[category]
            category_dir = os.path.join(directory, str(original_index))
            if os.path.exists(category_dir):
                for img_name in os.listdir(category_dir):
                    # Encode the image path to bytes
                    image_paths.append(os.path.join(category_dir, img_name).encode('utf-8'))
                    labels.append(i) # Use the new index based on the selected categories

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    if shuffle:
        dataset = dataset.shuffle(buffer_size=1000)

    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

    dataset = dataset.batch(batch_size)

    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

    return dataset

train_dir = "/content/drive/MyDrive/pest_classifier/classification/train"
validation_dir = "/content/drive/MyDrive/pest_classifier/classification/val"
test_dir = "/content/drive/MyDrive/pest_classifier/classification/test"

# Use the selected category names to create the datasets
train_dataset = create_dataset(train_dir, selected_category_names)
val_dataset = create_dataset(validation_dir, selected_category_names, shuffle=False)
test_dataset = create_dataset(test_dir, selected_category_names, shuffle=False)

print("tf.data.Dataset created for train, validation, and test sets with selected categories.")

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

resnet_checkpoint_callback = ModelCheckpoint(
    filepath='/content/drive/MyDrive/best_resnet_model.weights.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

mobilenet_checkpoint_callback = ModelCheckpoint(
    filepath='/content/drive/MyDrive/best_mobilenet_model.weights.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

inceptionv3_checkpoint_callback = ModelCheckpoint(
    filepath='/content/drive/MyDrive/best_inceptionv3_model.weights.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

early_stopping_callback = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    mode='max',
    restore_best_weights=True,
    verbose=1
)

TypeError: in user code:

    File "/tmp/ipython-input-33-667702121.py", line 7, in load_and_preprocess_image  *
        img = tf.io.read_file(image_path)

    TypeError: Input 'filename' of 'ReadFile' Op has type float32 that does not match expected type of string.


In [32]:
gc.collect()

print("Training ResNet model...")
resnet_history = resnet_model.fit(
    train_dataset,
    epochs=3,
    validation_data=val_dataset,
    callbacks=[resnet_checkpoint_callback, early_stopping_callback]
)


tf.keras.backend.clear_session()
gc.collect()


print("\nTraining MobileNetV2 model...")
mobilenet_history = mobilenet_model.fit(
    train_dataset,
    epochs=3,
    validation_data=val_dataset,
    callbacks=[mobilenet_checkpoint_callback, early_stopping_callback]
)

tf.keras.backend.clear_session()
gc.collect()


print("\nTraining InceptionV3 model...")
inceptionv3_history = inceptionv3_model.fit(
    train_dataset,
    epochs=3,
    validation_data=val_dataset,
    callbacks=[inceptionv3_checkpoint_callback, early_stopping_callback]
)

Training ResNet model...
Epoch 1/3


UnimplementedError: {{function_node __wrapped__MakeIterator_device_/job:localhost/replica:0/task:0/device:CPU:0}} Cast float to string is not supported
	 [[{{node Cast}}]] [Op:MakeIterator] name: 

In [ ]:

print("\nEvaluating ResNet model on the test dataset...")
resnet_test_results = resnet_model.evaluate(test_dataset)

print(f"ResNet Test Loss: {resnet_test_results[0]:.4f}")
print(f"ResNet Test Accuracy: {resnet_test_results[1]:.4f}")

print("\nEvaluating MobileNetV2 model on the test dataset...")
mobilenet_test_results = mobilenet_model.evaluate(test_dataset)

print(f"MobileNetV2 Test Loss: {mobilenet_test_results[0]:.4f}")
print(f"MobileNetV2 Test Accuracy: {mobilenet_test_results[1]:.4f}")

print("\nEvaluating InceptionV3 model on the test dataset...")
inceptionv3_test_results = inceptionv3_model.evaluate(test_dataset)

print(f"InceptionV3 Test Loss: {inceptionv3_test_results[0]:.4f}")
print(f"InceptionV3 Test Accuracy: {inceptionv3_test_results[1]:.4f}")